<a href="https://colab.research.google.com/github/KaichengDING/Triple-Defense/blob/main/idl_project_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Import Dependecies

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

import matplotlib.pyplot as plt
import time
import logging
import datetime

cuda = torch.cuda.is_available()
cuda

True

# Class and Function Definitions

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=1.0):
      self.std = std
      self.mean = mean
        
    def __call__(self, tensor):
      noise = torch.randn(tensor.size()) * self.std + self.mean
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
      return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)


class AddUniformNoise(object):
    def __init__(self, rg):
      self.rg = rg
        
    def __call__(self, tensor):
      # adding uniform noise [-range, range]
      noise = (torch.rand(tensor.size()) * 2.0 - 1.0) * self.rg
      res = tensor + noise
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
        return self.__class__.__name__ + '(range=[-{}, {}])'.format(self.rg, self.rg)


# Model definition
class ShuffleNet(nn.Module):
    def __init__(self, nb_classes =10):
        super(ShuffleNet, self).__init__()
        self.shuffle = models.shufflenet_v2_x2_0()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.shuffle(x)
        x = self.linear(x)
        return x


# training and testing functions
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader): 
            data = data.to(device)
            target = target.to(device)

            with torch.cuda.amp.autocast():
              outputs = model(data)

              _, predicted = torch.max(outputs.data, 1)
              total_predictions += target.size(0)
              correct_predictions += (predicted == target).sum().item()

              loss = criterion(outputs, target).detach()
              running_loss += loss.item()

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

# Configure Logging

In [ ]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('/content/gdrive/My Drive/IDL_Project/logs/log_{}.log.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

# Hyper Parameter Settings

In [ ]:
train_batchsize = 128
test_batchsize = 100
num_workers = 8

GAUSSIAN = 'g'
UNIFORM = 'u'

img_size = 224
num_classes = 10

n_epochs = 30
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4

num_models = 10
start_seed = 0
seeds = np.arange(start_seed, start_seed + num_models)

noise_type = UNIFORM
noise_intensity = 0.1

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 
                'min_lr': min_lr, 
                'weight_decay': weight_decay, 
                'num_epochs': n_epochs, 
                'noise_type': noise_type, 
                'noise_intensity': noise_intensity}
logging.info(hyper_params)


2020-12-10 23:29:15,376 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_epochs': 30, 'noise_type': 'u', 'noise_intensity': 0.1}


# Kick off Training

In [ ]:
for seed in seeds:
  logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)
  
  if noise_type == GAUSSIAN:
    train_transform = transforms.Compose([transforms.Resize(size=img_size),
                                          transforms.ColorJitter(hue=.05, saturation=.05),
                                          transforms.RandomHorizontalFlip(p=0.5),
                                          transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                          AddGaussianNoise(0., noise_intensity)])

    test_transform = transforms.Compose([transforms.Resize(size=img_size),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                         AddGaussianNoise(0., noise_intensity)])
  elif noise_type == UNIFORM:
    train_transform = transforms.Compose([transforms.Resize(size=img_size),
                                          transforms.ColorJitter(hue=.05, saturation=.05),
                                          transforms.RandomHorizontalFlip(p=0.5),
                                          transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                          AddUniformNoise(noise_intensity)])

    test_transform = transforms.Compose([transforms.Resize(size=img_size),
                                         transforms.ToTensor(),
                                         transforms.Normalize((0, 0, 0), (1, 1, 1)),
                                         AddUniformNoise(noise_intensity)]) 
  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  model = ShuffleNet(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  for i in range(n_epochs):
    tic = time.time()
    logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    toc = time.time()
    logging.info('Time: {}s'.format(toc - tic))
    logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/content/gdrive/My Drive/IDL_Project/modelS/Shufflenet{}_{}_{}".format(seed, noise_type, noise_intensity))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

2020-12-10 23:29:41,403 INFO Seed: 0


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified



2020-12-10 23:29:58,521 INFO Epoch: 0
2020-12-10 23:33:16,332 INFO Training Loss: 1.8978542424833682
2020-12-10 23:33:16,333 INFO Training Accuracy: 29.584%
2020-12-10 23:33:30,758 INFO Testing Loss: 1.7456210052967072
2020-12-10 23:33:30,760 INFO Testing Accuracy: 37.18%
2020-12-10 23:33:30,761 INFO Time: 212.23989868164062s
2020-12-10 23:33:30,761 INFO ====================
2020-12-10 23:33:30,762 INFO Epoch: 1
2020-12-10 23:36:49,251 INFO Training Loss: 1.6294291571278097
2020-12-10 23:36:49,253 INFO Training Accuracy: 39.894%
2020-12-10 23:37:04,017 INFO Testing Loss: 1.5566447019577025
2020-12-10 23:37:04,018 INFO Testing Accuracy: 43.14%
2020-12-10 23:37:04,019 INFO Time: 213.25716996192932s
2020-12-10 23:37:04,021 INFO ====================
2020-12-10 23:37:04,029 INFO Epoch: 2
2020-12-10 23:40:23,303 INFO Training Loss: 1.5007278678362326
2020-12-10 23:40:23,304 INFO Training Accuracy: 44.666%
2020-12-10 23:40:38,023 INFO Testing Loss: 1.4106473290920258
2020-12-10 23:40:38,025 I

Epoch    25: reducing learning rate of group 0 to 1.0000e-05.


2020-12-11 01:03:39,908 INFO Training Loss: 0.2694124300080492
2020-12-11 01:03:39,910 INFO Training Accuracy: 91.024%
2020-12-11 01:03:54,497 INFO Testing Loss: 0.555100214779377
2020-12-11 01:03:54,499 INFO Testing Accuracy: 82.17%
2020-12-11 01:03:54,500 INFO Time: 212.8791642189026s
2020-12-11 01:03:54,502 INFO ====================
2020-12-11 01:03:54,503 INFO Epoch: 26
2020-12-11 01:07:10,170 INFO Training Loss: 0.24423068087271718
2020-12-11 01:07:10,172 INFO Training Accuracy: 91.716%
2020-12-11 01:07:24,644 INFO Testing Loss: 0.5523957930505276
2020-12-11 01:07:24,645 INFO Testing Accuracy: 82.05%
2020-12-11 01:07:24,646 INFO Time: 210.14380550384521s
2020-12-11 01:07:24,647 INFO ====================
2020-12-11 01:07:24,649 INFO Epoch: 27
2020-12-11 01:10:39,304 INFO Training Loss: 0.22774018339641258
2020-12-11 01:10:39,305 INFO Training Accuracy: 92.378%
2020-12-11 01:10:53,708 INFO Testing Loss: 0.5599207966029645
2020-12-11 01:10:53,709 INFO Testing Accuracy: 82.61%
2020-12

...

2020-12-11 16:16:21,694 INFO Training Loss: 0.22000508267632532
2020-12-11 16:16:21,696 INFO Training Accuracy: 92.574%
2020-12-11 16:16:37,122 INFO Testing Loss: 0.5564506837725639
2020-12-11 16:16:37,123 INFO Testing Accuracy: 82.75%
2020-12-11 16:16:37,124 INFO Time: 218.651198387146s
2020-12-11 16:16:37,126 INFO ====================
2020-12-11 16:16:37,127 INFO Epoch: 28
2020-12-11 16:20:02,571 INFO Training Loss: 0.18969703593369944
2020-12-11 16:20:02,573 INFO Training Accuracy: 93.738%
2020-12-11 16:20:18,224 INFO Testing Loss: 0.5573269364237785
2020-12-11 16:20:18,225 INFO Testing Accuracy: 82.8%
2020-12-11 16:20:18,226 INFO Time: 221.09901404380798s
2020-12-11 16:20:18,227 INFO ====================
2020-12-11 16:20:18,230 INFO Epoch: 29
2020-12-11 16:23:44,369 INFO Training Loss: 0.1777760830071881
2020-12-11 16:23:44,371 INFO Training Accuracy: 94.164%
2020-12-11 16:24:00,362 INFO Testing Loss: 0.568842182457447
2020-12-11 16:24:00,364 INFO Testing Accuracy: 82.71%
2020-12-1

Files already downloaded and verified
Files already downloaded and verified


2020-12-11 16:24:02,712 INFO Epoch: 0
2020-12-11 16:27:31,793 INFO Training Loss: 1.9225136404452117
2020-12-11 16:27:31,795 INFO Training Accuracy: 28.449999999999996%
2020-12-11 16:27:47,071 INFO Testing Loss: 1.6878552114963532
2020-12-11 16:27:47,074 INFO Testing Accuracy: 37.3%
2020-12-11 16:27:47,075 INFO Time: 224.36287784576416s
2020-12-11 16:27:47,077 INFO ====================
2020-12-11 16:27:47,079 INFO Epoch: 1
2020-12-11 16:31:11,429 INFO Training Loss: 1.6548052303626408
2020-12-11 16:31:11,431 INFO Training Accuracy: 38.83%
2020-12-11 16:31:26,658 INFO Testing Loss: 1.5631058621406555
2020-12-11 16:31:26,660 INFO Testing Accuracy: 41.36%
2020-12-11 16:31:26,661 INFO Time: 219.58226656913757s
2020-12-11 16:31:26,664 INFO ====================
2020-12-11 16:31:26,666 INFO Epoch: 2
2020-12-11 16:34:48,972 INFO Training Loss: 1.5230565159522054
2020-12-11 16:34:48,974 INFO Training Accuracy: 44.076%
2020-12-11 16:35:03,968 INFO Testing Loss: 1.5145982336997985
2020-12-11 16:3

Epoch    27: reducing learning rate of group 0 to 1.0000e-05.


2020-12-11 18:05:06,817 INFO Training Loss: 0.23619105912687832
2020-12-11 18:05:06,819 INFO Training Accuracy: 92.08800000000001%
2020-12-11 18:05:22,766 INFO Testing Loss: 0.5385158902406693
2020-12-11 18:05:22,767 INFO Testing Accuracy: 82.86%
2020-12-11 18:05:22,769 INFO Time: 228.6109540462494s
2020-12-11 18:05:22,771 INFO ====================
2020-12-11 18:05:22,772 INFO Epoch: 28
2020-12-11 18:08:55,510 INFO Training Loss: 0.20473499495126402
2020-12-11 18:08:55,512 INFO Training Accuracy: 93.282%
2020-12-11 18:09:11,559 INFO Testing Loss: 0.5371231456100941
2020-12-11 18:09:11,560 INFO Testing Accuracy: 83.06%
2020-12-11 18:09:11,562 INFO Time: 228.79004669189453s
2020-12-11 18:09:11,563 INFO ====================
2020-12-11 18:09:11,565 INFO Epoch: 29
2020-12-11 18:12:46,640 INFO Training Loss: 0.1920565630849975
2020-12-11 18:12:46,642 INFO Training Accuracy: 93.688%
2020-12-11 18:13:02,678 INFO Testing Loss: 0.5427057467401027
2020-12-11 18:13:02,679 INFO Testing Accuracy: 83